In [ ]:
import Levenshtein as lev
import requests
import json
import math
import numpy as np
import time
import pandas as pd
from secrets import *

In [2]:
post_request = requests.post(url = 'https://id.twitch.tv/oauth2/token', params = {
    'client_id':client_id, 'client_secret':client_secret, 'grant_type': 'client_credentials'
})

In [3]:
access_token = json.loads(post_request.text)['access_token']

In [ ]:
# gather igdb data
game_list = []
for i in range(430):
    data = requests.post(url = 'https://api.igdb.com/v4/games', headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'limit': '500', 'offset': f"{i*500}", 'fields': '*'})
    temp_list = json.loads(data.text)
    game_list.extend(temp_list)
    if i % 5 == 0:
        time.sleep(1)
    print(i)

In [ ]:
# making igdb csv
game_list_np = np.array(game_list)
DF = pd.json_normalize(game_list_np)
DF.head()
DF.to_csv('igdb_games.csv') 

In [ ]:
# building csv of rawg databse
rawg_game_list = []
for i in range (0,20025): 
    data = requests.get(url = 'https://api.rawg.io/api/games', params={'key':rawg_api_key,'page':i,'page_size':'40'})
    rawg_temp_list = json.loads(data.text)['results']
    rawg_game_list.extend(rawg_temp_list)
    if (i % 500 == 0):
        print(i)
        rawg_game_list_np = np.array(rawg_game_list)
        DF = pd.json_normalize(rawg_game_list_np)
        DF.to_csv(f"rawg_games_{i}.csv")    

In [ ]:
# creating full csv file
DF1 = pd.read_csv('rawg_games_10000.csv')
DF1 = DF1.drop('Unnamed: 0', axis = 1)
DF1 = DF1.drop_duplicates()
DF2 = pd.read_csv('rawg_games_20024.csv').drop('Unnamed: 0', axis = 1).drop_duplicates()
DF1.columns
DF2.columns
df_full = pd.concat([DF1, DF2], axis = 0)
df_full = df_full.drop_duplicates()
df_full.columns
df_full = df_full.reset_index()
df_full = df_full.drop('index', axis = 1)
df_full.to_csv('rawg_games_full.csv')

In [ ]:
#load full datasets
igdb_games_df = pd.read_csv('igdb_games.csv', index_col=[0])
rawg_games_df = pd.read_csv('rawg_games_full.csv', index_col=[0])

C:\Users\Grayson Kornberg\AppData\Local\Temp\ipykernel_9868\924709412.py:1: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_games_df = pd.read_csv('igdb_games.csv', index_col=[0])


In [55]:
#unneeded columns
igdb_drop_fields = ['category','created_at','updated_at','checksum','involved_companies','player_perspectives','themes','websites','game_engines','keywords','status','collection','franchises','videos', 'version_parent', 'version_title','aggregated_rating', 'aggregated_rating_count','alternative_names', 'follows', 'franchise','rating','rating_count','ports','artworks', 'storyline', 'hypes', 'multiplayer_modes', 'game_localizations', 'parent_game', 'bundles', 'standalone_expansions', 'remasters' ,'remakes', 'expansions', 'expanded_games', 'dlcs', 'forks','external_games','release_dates','screenshots','game_modes']
rawg_drop_fields = ['tba','rating_top','ratings','reviews_text_count','metacritic','playtime','suggestions_count','updated','user_game','reviews_count','saturated_color','dominant_color','parent_platforms','stores','clip','short_screenshots','esrb_rating.id','esrb_rating.slug','esrb_rating','community_rating','added_by_status','added']

In [12]:
#keeps track of already found ids in api calls
themes = {}
keywords = {}
games = {}
player_perspectives = {}
genres = {}
platforms = {}
age_ratings = {}
covers = {}

In [29]:
#functions for transforming both datasets to have similar data/do api calls
def genres_str_to_list(str):
    genre_list = []
    for i in eval(str):
        genre_list.append(i['name'])
    return genre_list
def genre_id_list_to_str_list(ids):
    if (type(ids) == float and math.isnan(ids)):
        return None
    genre_list = []
    for id in eval(ids):
        if (id in genres):
            genre_list.append(genres[id])
        else:
            data = requests.post(url = 'https://api.igdb.com/v4/genres', headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'fields': f"name; where id = ({id})"})
            name = json.loads(data.text)[0]['name']
            genre_list.append(name)
            genres[id] = name
    return genre_list
def platform_str_to_list(platforms):
    platform_list = []
    for platform in eval(platforms):
        platform_list.append(platform['platform']['name'])
    return platform_list
def platform_id_to_list(ids):
    if (type(ids) == float and math.isnan(ids)):
        return None
    platform_list = []
    for id in eval(ids):
        if id in platforms:
            platform_list.append(platforms[id])
        else:
            data = requests.post(url = 'https://api.igdb.com/v4/platforms', headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'fields': f"name; where id = ({id})"})
            name = json.loads(data.text)
            if len(name) > 0:
                name = name[0]['name']
                platform_list.append(name)
                platforms[id] = name
    return platform_list
def tag_str_to_tags_list(tags):
    tag_list = []
    for tag in eval(tags):
        tag_list.append(tag['name'])
    return tag_list
def tag_hash_to_tag_list(tags):
    if (type(tags) == float and math.isnan(tags)):
        return None
    if tags == None:
        return None
    tags_list = []
    tags_ids = {0:[],1:[],2:[],3:[],4:[]}
    types = {0:'themes',1:'genres',2:'keywords',3:'games',4:'player_perspectives'}
    for tag in eval(tags):
        keywordID = tag & (1 << 28) - 1
        keywordTypeID = ((tag >> 28) & 15)
        if keywordID in eval(types[keywordTypeID]):
            tags_list.append(eval(types[keywordTypeID])[keywordID])
        else:
            tags_ids[keywordTypeID].append(str(keywordID))
    for i in range(5):
        if len(tags_ids[i]) > 0:
            id_str = ','.join(tags_ids[i])
            data = requests.post(url = f"https://api.igdb.com/v4/{types[i]}", headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'fields': f"name; where id = ({id_str})"})
            name = json.loads(data.text)
            for tag_name in name:
                tags_list.append(tag_name['name'])
                eval(types[i])[int(tag_name['id'])] = tag_name['name']
    return tags_list
ratings_enum = {1:'Early Childhood',2:'Everyone',3:'Everyone 10+',4:'Teen',5:'Mature',6:'Rating Pending',7:'Early Childhood',8:'Everyone',9:'Everyone 10+',10:'Teen',11:'Mature',12:'Adults Only',13:'Everyone',14:'Everyone 10+',15:'Teen',16:'Teen',17:'Mature',18:'Everyone',19:'Early Childhood',20:'Everyone 10+',21:'Mature',22:'Everyone',23:'Everyone 10+',24:'Teen',25: 'Mature',26:'Rating Pending',27:'Everyone',28:'Everyone 10+',29:'Everyone 10+',30:'Teen',31:'Teen',32:'Mature',33:'Everyone',34:'Everyone 10+',35:'Teen',36:'Mature',37:'Mature',38:'Rating Pending'}
def age_rating_str_to_list(age_rating_str):
    if (type(age_rating_str) == float and math.isnan(age_rating_str)):
        return ['Rating Pending']
    age_rating_number = eval(age_rating_str)[0]
    if age_rating_number in age_ratings:
        return age_ratings[age_rating_number]
    data = requests.post(url = f"https://api.igdb.com/v4/age_ratings", headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'fields': f"rating; where id = ({age_rating_number})"})
    rating_name = json.loads(data.text)
    rating_name = rating_name[0] if len(rating_name) > 0 else {'rating':39}
    if rating_name['rating'] > 38:
        return 'Rating Pending'
    else:
        age_ratings[int(rating_name['id'])] = ratings_enum[rating_name['rating']]
        return ratings_enum[rating_name['rating']]
def cover_id_to_uri(cover_id):
    if (math.isnan(cover_id)):
        return None
    cover_id = int(cover_id)
    if cover_id in covers:
        return covers[cover_id]
    data = requests.post(url = f"https://api.igdb.com/v4/covers", headers={'Client-ID':client_id, 'Authorization':f"Bearer {access_token}"}, params={'fields': f"url; where id = ({cover_id})"})
    url = json.loads(data.text)
    if (len(url) > 0):
        url = url[0]['url']
        covers[cover_id] = 'https:' + url
        return 'https:' + url
    else:
        return None

In [680]:
#apply functions to datasets to transform them
igdb_games_df_reduced = igdb_games_df.drop(igdb_drop_fields, axis = 1)
rawg_games_df_reduced = rawg_games_df.drop(rawg_drop_fields, axis = 1)
rawg_games_df_reduced = rawg_games_df_reduced[rawg_games_df_reduced['ratings_count'] != 0].reset_index()
igdb_games_df_reduced = igdb_games_df_reduced[igdb_games_df_reduced['total_rating_count'].notnull()].reset_index()
igdb_games_df_reduced['released'] = pd.to_datetime(igdb_games_df_reduced['first_release_date'], unit = 's').dt.strftime('%Y-%m-%d')
igdb_games_df_reduced = igdb_games_df_reduced.drop('first_release_date', axis = 1)
igdb_games_df_reduced = igdb_games_df_reduced.drop('game_modes', axis = 1)
rawg_games_df_reduced['genres'] = rawg_games_df_reduced['genres'].apply(genres_str_to_list)
igdb_games_df_reduced['genres'] = igdb_games_df_reduced['genres'].apply(genre_id_list_to_str_list)
rawg_games_df_reduced['platforms'] = rawg_games_df_reduced['platforms'].apply(platform_str_to_list)
igdb_games_df_reduced['platforms'] = igdb_games_df_reduced['platforms'].apply(platform_id_to_list)
rawg_games_df_reduced['tags'] = rawg_games_df_reduced['tags'].apply(tag_str_to_tags_list)
igdb_games_df_reduced['tags'] = igdb_games_df_reduced['tags'].apply(tag_hash_to_tag_list)
igdb_games_df_reduced['age_ratings'] = igdb_games_df_reduced['age_ratings'].apply(age_rating_str_to_list)
rawg_games_df_reduced['rating'] = rawg_games_df_reduced['rating'] * 20
igdb_games_df_reduced['cover'] = igdb_games_df_reduced['cover'].apply(cover_id_to_uri)
igdb_games_df_reduced.to_csv('igdb_games_reduced.csv')
rawg_games_df_reduced.to_csv('rawg_games_reduced.csv')

In [4]:
#load the reduced datasets and steamdb and similarities
rawg_games_df_reduced = pd.read_csv('rawg_games_reduced.csv',index_col=[0])
igdb_games_df_reduced = pd.read_csv('igdb_games_reduced.csv',index_col=[0])
igdb_games_df = pd.read_csv('igdb_games.csv', index_col=[0])
steamdb_df = pd.read_csv('steamdb-scrape.csv', index_col=[0])
similarities_df = pd.read_csv('similarities.csv', index_col=[0])
more_similarities_df = pd.read_csv('more_similarities.csv', index_col=[0])
combined_df = pd.read_csv('combined_df.csv', index_col=[0])

C:\Users\Grayson Kornberg\AppData\Local\Temp\ipykernel_7844\404466041.py:4: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_games_df = pd.read_csv('igdb_games.csv', index_col=[0])
C:\Users\Grayson Kornberg\AppData\Local\Temp\ipykernel_7844\404466041.py:8: DtypeWarning: Columns (10,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('combined_df.csv', index_col=[0])


In [212]:
# find rawg games that are in igdb database
rawg_reduced_games_in_full_igdb = rawg_games_df_reduced[rawg_games_df_reduced.slug.isin(igdb_games_df['slug'])]['slug']
igdb_games_df_rawg_games_only = igdb_games_df[igdb_games_df['slug'].isin(rawg_reduced_games_in_full_igdb)]

32347

In [274]:
# use igdb database to populate rawg columns with matching games
def insert_similar_games(slug):
    if type(slug) == float and math.isnan(slug):
        return None
    if slug in rawg_reduced_games_in_full_igdb.values:
        similar_games = igdb_games_df_rawg_games_only[igdb_games_df_rawg_games_only['slug'] == slug]['similar_games'].values[0]
        if type(similar_games) == float and math.isnan(similar_games):
            return None
        return similar_games
rawg_games_df_reduced['similar_games'] = rawg_games_df_reduced['slug']
rawg_games_df_reduced['similar_games'] = rawg_games_df_reduced['similar_games'].apply(insert_similar_games)
def insert_urls(slug):
    if type(slug) == float and math.isnan(slug):
        return None
    if slug in rawg_reduced_games_in_full_igdb.values:
        url = igdb_games_df_rawg_games_only[igdb_games_df_rawg_games_only['slug'] == slug]['url'].values[0]
        if type(url) == float and math.isnan(url):
            return None
        return url
rawg_games_df_reduced['url'] = rawg_games_df_reduced['slug']
rawg_games_df_reduced['url'] = rawg_games_df_reduced['url'].apply(insert_urls)
def insert_summaries(slug):
    if type(slug) == float and math.isnan(slug):
        return None
    if slug in rawg_reduced_games_in_full_igdb.values:
        summary = igdb_games_df_rawg_games_only[igdb_games_df_rawg_games_only['slug'] == slug]['summary'].values[0]
        if type(summary) == float and math.isnan(summary):
            return None
        return summary
rawg_games_df_reduced['summary'] = rawg_games_df_reduced['slug']
rawg_games_df_reduced['summary'] = rawg_games_df_reduced['summary'].apply(insert_summaries)

In [16]:
# transform similar ids columns to show games instead of ids
def similar_games_ids_to_games_list(games_ids):
    if (type(games_ids) == float and math.isnan(games_ids)):
        return None
    games_list = []
    for id in eval(games_ids):
        game = igdb_games_df[igdb_games_df['id'] == id]['name'].values
        if len(game) > 0:
            games_list.append(game[0])
    return games_list
igdb_games_df_reduced['similar_games'] = igdb_games_df_reduced['similar_games'].apply(similar_games_ids_to_games_list)
rawg_games_df_reduced['similar_games'] = rawg_games_df_reduced['similar_games'].apply(similar_games_ids_to_games_list)

In [158]:
# these sets contain the different category names for these columns
genres_set_rawg = set()
genres_set_igdb = set()
platforms_set_rawg = set()
platforms_set_igdb = set()
age_ratings_set_rawg = set()
age_ratings_set_igdb = set()
for age_rating in igdb_games_df_reduced['age_rating'].values:
    age_ratings_set_igdb.add(age_rating)
for age_rating in rawg_games_df_reduced['age_rating'].values:
    age_ratings_set_rawg.add(age_rating)
for genre_list in rawg_games_df_reduced['genres'].values:
    for genre in genre_list:
        genres_set_rawg.add(genre)
for genre_list in igdb_games_df_reduced['genres'].values:
        if type(genre_list) == list:
            for genre in genre_list:
                genres_set_igdb.add(genre)
for platform_list in rawg_games_df_reduced['platforms'].values:
    for platform in platform_list:
        platforms_set_rawg.add(platform)
for platform_list in igdb_games_df_reduced['platforms'].values:
        if type(platform_list) == list:
            for platform in platform_list:
                platforms_set_igdb.add(platform)

In [32]:
# adjust age_ratings, genres, and platforms to be similar for both datasets
def age_ratings_adjust(age_rating_list):
    if type(age_rating_list) == float:
        return None
    for age_rating in eval(age_rating_list):
        return age_rating
def genre_change(genre_list):
    if type(genre_list) == float and math.isnan(genre_list):
        return None
    genres = []
    for genre in genre_list:
        if genre == 'Board Games' or genre == 'Card':
            genres.append('Card & Board Game')
        elif genre == 'Platformer':
            genres.append('Platform')
        elif genre == 'Role-playing (RPG)':
            genres.append('RPG')
        else:
            genres.append(genre)
    return genres
def platform_change(platform_list):
    if type(platform_list) == float and math.isnan(platform_list):
        return None
    if platform_list == None:
        return None
    platforms = []
    for platform in (platform_list):
        if platform == '3DO Interactive Multiplayer':
            platforms.append('3DO')
        elif platform == 'Atari ST':
            platforms.append('Atari ST/STE')
        elif platform == 'Classic Macintosh':
            platforms.append('Mac')
        elif platform == 'Amiga':
            platforms.append('Commodore / Amiga')
        elif platform == 'Amiga CD32':
            platforms.append('Commodore / Amiga')
        elif platform == 'Commodore 16':
            platforms.append('Commodore / Amiga')
        elif platform == 'Commodore C64/128/MAX':
            platforms.append('Commodore / Amiga')
        elif platform == 'Commodore CDTV':
            platforms.append('Commodore / Amiga')
        elif platform == 'Commodore Plus/4':
            platforms.append('Commodore / Amiga')
        elif platform == 'Commodore VIC-20':
            platforms.append('Commodore / Amiga')
        elif platform == 'Game Gear':
            platforms.append('Sega Game Gear')
        elif platform == 'GameCube':
            platforms.append('Nintendo GameCube')
        elif platform == 'Genesis':
            platforms.append('Sega Mega Drive/Genesis')
        elif platform == 'Jaguar':
            platforms.append('Atari Jaguar')
        elif platform == 'Nintendo Entertainment System':
            platforms.append('NES')
        elif 'Neo Geo' in platform:
            platforms.append('Neo Geo')
        elif platform == "PC (Microsoft Windows)":
            platforms.append('PC')
        elif platform == "PS Vita":
            platforms.append('PlayStation Vita')
        elif platform == "PlayStation Portable":
            platforms.append('PSP')
        elif platform == "SEGA 32X":
            platforms.append('Sega 32X')
        elif platform == "SEGA CD":
            platforms.append('Sega CD')
        elif platform == "SEGA Master System":
            platforms.append('Sega Master System/Mark III')
        elif platform == "SEGA Saturn":
            platforms.append('Sega Saturn')
        elif platform == "Super Nintendo Entertainment System":
            platforms.append('SNES')
        elif platform == "Web":
            platforms.append('Web browser')
        elif platform == "Xbox Series S/X":
            platforms.append('Xbox Series X|S')
        else:
            platforms.append(platform)
    return platforms

In [ ]:
igdb_games_df_reduced['age_rating'] = igdb_games_df_reduced['age_rating'].apply(age_ratings_adjust)
igdb_games_df_reduced['genres'] = igdb_games_df_reduced['genres'].apply(genre_change)
rawg_games_df_reduced['genres'] = rawg_games_df_reduced['genres'].apply(genre_change)
igdb_games_df_reduced['platforms'] = igdb_games_df_reduced['platforms'].apply(platform_change)
rawg_games_df_reduced['platforms'] = rawg_games_df_reduced['platforms'].apply(platform_change)

In [630]:
# enumerate duplicate names so names are unique
duplicate_names = np.unique(igdb_games_df_reduced['name'].values, return_counts=True)[0][(np.unique(igdb_games_df_reduced['name'].values, return_counts=True)[1] != 1)]
for i in duplicate_names:
    duplicate_name_pair = igdb_games_df_reduced[igdb_games_df_reduced['name'] == i]
    for j in range(duplicate_name_pair.shape[0]):
        igdb_games_df_reduced.at[duplicate_name_pair.index[j],'name'] = i + f"-{j}"

In [595]:
# change slugs and names to be equal between datasets if one or is equal
def adjust_slugs(row):
    if row['name'] in (igdb_games_df_reduced['name'].values):
        series_match = igdb_games_df_reduced[igdb_games_df_reduced['name'] == row['name']]
        if series_match['release_date'].values[0] == row['release_date']:
            return series_match['slug'].values[0]
        else:
            return row['slug']
    else:
        return row['slug']
def adjust_names(row):
    if row['slug'] in (igdb_games_df_reduced['slug'].values):
        series_match = igdb_games_df_reduced[igdb_games_df_reduced['slug'] == row['slug']]
        if series_match['release_date'].values[0] == row['release_date']:
            return series_match['name'].values[0]
        else:
            return row['name']
    else:
        return row['name']
rawg_games_df_reduced['new_slugs'] = rawg_games_df_reduced.apply(adjust_slugs, axis = 1)
rawg_games_df_reduced['new_names'] = rawg_games_df_reduced.apply(adjust_names, axis = 1)

In [66]:
# get games that overlap between both datasets
igdb_df_overlap = igdb_games_df_reduced[igdb_games_df_reduced['slug'].isin(rawg_games_df_reduced['slug']) | igdb_games_df_reduced['name'].isin(rawg_games_df_reduced['name'])]
rawg_df_overlap = rawg_games_df_reduced[rawg_games_df_reduced['slug'].isin(igdb_games_df_reduced['slug']) | rawg_games_df_reduced['name'].isin(igdb_games_df_reduced['name'])]

In [686]:
# if same slug or name appears twice in a dataset, add hyphen to differentiate them because they are actually different games
for i in (np.unique(np.append(igdb_df_overlap['slug'].values, rawg_df_overlap['new_slugs'].values), return_counts=True)[0])[(np.unique(np.append(igdb_df_overlap['slug'].values, rawg_df_overlap['new_slugs'].values), return_counts=True)[1] != 2)]:
    if i in rawg_df_overlap['new_slugs'].values:
        rawg_games_df_reduced.at[rawg_games_df_reduced[rawg_games_df_reduced['new_slugs'] == i].index[0],'new_names'] = i+'-'
    else:
        igdb_games_df_reduced.at[igdb_games_df_reduced[igdb_games_df_reduced['slug'] == i].index[0],'name'] = i+'-'
for i in (np.unique(np.append(igdb_df_overlap['name'].values, rawg_df_overlap['new_names'].values), return_counts=True)[0])[(np.unique(np.append(igdb_df_overlap['name'].values, rawg_df_overlap['new_names'].values), return_counts=True)[1] != 2)]:
    if i in rawg_df_overlap['new_names'].values:
        rawg_games_df_reduced.at[rawg_games_df_reduced[rawg_games_df_reduced['new_names'] == i].index[0],'new_slugs'] = i+'-'
    else:
        igdb_games_df_reduced.at[igdb_games_df_reduced[igdb_games_df_reduced['name'] == i].index[0],'slug'] = i+'-'

In [704]:
#replace slugs and names with new names
rawg_games_df_reduced['slug'] = rawg_games_df_reduced['new_slugs']
rawg_games_df_reduced['name'] = rawg_games_df_reduced['new_names']
rawg_games_df_reduced = rawg_games_df_reduced.drop(['new_slugs','new_names'], axis = 1)

In [713]:
# create adjusted ratings from overlap
overlap_adjusted_ratings = pd.DataFrame(columns=['slug','rating','rating_count'])
overlap_adjusted_ratings['slug'] = igdb_df_overlap['slug']
overlap_adjusted_ratings['rating'] = igdb_df_overlap['slug']
overlap_adjusted_ratings['ratings_count'] = igdb_df_overlap['slug']
def adjust_ratings(slug):
    rawg_rating = rawg_df_overlap[rawg_df_overlap['slug'] == slug]['rating'].values[0]
    igdb_rating = igdb_df_overlap[igdb_df_overlap['slug'] == slug]['rating'].values[0]
    rawg_rating_count = rawg_df_overlap[rawg_df_overlap['slug'] == slug]['ratings_count'].values[0]
    igdb_rating_count = igdb_df_overlap[igdb_df_overlap['slug'] == slug]['ratings_count'].values[0]
    if (rawg_rating_count == 0):
        rawg_rating_count = 1
    if igdb_rating_count == 0:
        igdb_rating_count = 1
    return rawg_rating if rawg_rating_count >= igdb_rating_count  and rawg_rating != 0 else igdb_rating
def adjust_ratings_counts(slug):
    rawg_rating_count = rawg_df_overlap[rawg_df_overlap['slug'] == slug]['ratings_count'].values[0]
    igdb_rating_count = igdb_df_overlap[igdb_df_overlap['slug'] == slug]['ratings_count'].values[0]
    return rawg_rating_count + igdb_rating_count if igdb_rating_count != 0 else rawg_rating_count + igdb_rating_count + 1

In [ ]:
overlap_adjusted_ratings['rating'] = overlap_adjusted_ratings['rating'].apply(adjust_ratings)
overlap_adjusted_ratings['ratings_count'] = overlap_adjusted_ratings['ratings_count'].apply(adjust_ratings_counts)
overlap_adjusted_ratings.to_csv('overlap.csv')

In [3]:
overlap_adjusted_ratings = pd.read_csv('overlap.csv', index_col=[0])

In [8]:
# non overlapping games
igdb_df_not_overlap = igdb_games_df_reduced[~igdb_games_df_reduced['slug'].isin(rawg_games_df_reduced['slug'])]
rawg_df_not_overlap = rawg_games_df_reduced[~rawg_games_df_reduced['slug'].isin(igdb_games_df_reduced['slug'])]

In [ ]:
# create combined df
combined_df = pd.DataFrame(columns=['id_igdb','id_rawg','name','slug','rating','ratings_count','genres','image','platforms','release_date','tags','age_rating','similar_games','summary','url'])
combined_df = combined_df.append(igdb_df_not_overlap)
combined_df['id_igdb'] = combined_df['id']
combined_df = combined_df.drop(['id'], axis = 1)
combined_df = combined_df.reset_index().drop(['index'], axis = 1)
combined_df = combined_df.append(rawg_df_not_overlap)
combined_df['id_rawg'] = combined_df['id']
combined_df = combined_df.drop(['id'], axis = 1)
combined_df = combined_df.reset_index().drop(['index'], axis = 1)
igdb_df_overlap['rating'] = igdb_df_overlap['slug'].apply(lambda x: overlap_adjusted_ratings[overlap_adjusted_ratings['slug'] == x]['rating'].values[0])
igdb_df_overlap['ratings_count'] = igdb_df_overlap['slug'].apply(lambda x: overlap_adjusted_ratings[overlap_adjusted_ratings['slug'] == x]['ratings_count'].values[0])
combined_df = combined_df.append(igdb_df_overlap.rename({'id':'id_igdb'},axis = 1))
combined_df = combined_df.append(igdb_df_overlap.rename({'id':'id_igdb'},axis = 1))

In [154]:
combined_df = pd.read_csv('combined_df.csv', index_col=[0])

In [ ]:
# steamdb games that appear to not be in other datasets
steamdb_df_game_names_unique = steamdb_df[(~steamdb_df['name'].isin(igdb_games_df_reduced['name']) & ~steamdb_df['name'].isin(rawg_games_df_reduced['name']))]['name'].values

In [ ]:
# get unique names of games that are in combined dataset
rawg_and_igdb_game_names_reduced = np.append(igdb_df_not_overlap['name'].values, rawg_df_not_overlap['name'].values)
rawg_and_igdb_game_names_reduced = np.append(rawg_and_igdb_game_names_reduced, igdb_df_overlap['name'].values)

In [86]:
# create dataframe of similarities to see which steamdb games are actually not in datasets based on Levenshtein similar function
def similar(a,b):
    max_similarity = (-1, '')
    for i in b:
        similarity = lev.ratio(a, i)
        if (similarity > max_similarity[0]):
            max_similarity = (similarity, i)
    return max_similarity

In [ ]:
similarities = []
for i in steamdb_df_game_names_unique:
    similarities.append(similar(i,rawg_and_igdb_game_names_reduced))
similarities_name = np.array([x[1] for x in similarities])
similarities_value = np.array([x[0] for x in similarities])
similarities_df = pd.DataFrame({'original_name':steamdb_df_game_names_unique, 'matched_name':similarities_name,'value':similarities_value})
similarities_df.to_csv('similarities.csv')

In [182]:
# how many steamdb games are not in combined dataset
unique_steamdb_games = similarities_df[similarities_df['value'] < 0.75]

In [12]:
# unused games in igdb dataset that dont have ratings
good_unused_igdb_games = igdb_games_df[(igdb_games_df['total_rating'].isna()) & (~igdb_games_df['slug'].isin(combined_df['slug'])) & (~igdb_games_df['genres'].isna()) & (~igdb_games_df['summary'].isna()) & (~igdb_games_df['cover'].isna()) & (~igdb_games_df['id'].isin(combined_df['id_igdb']))]

In [90]:
# find which games in steamdb are in combined_df
steamdb_to_already_rated_games_similarities = []
for i in steamdb_df['name'].values:
    steamdb_to_already_rated_games_similarities.append(similar(i,combined_df['name'].values))
steamdb_to_already_rated_games_similarities_values = np.array(steamdb_to_already_rated_games_similarities)[:,0].astype(np.float64)
steamdb_to_already_rated_games_similarities_names = np.array(steamdb_to_already_rated_games_similarities)[:,1]
steamdb_to_already_rated_games_similarities_df = pd.DataFrame({'original_name':steamdb_df['name'].values, 'matched_name':steamdb_to_already_rated_games_similarities_names,'value':steamdb_to_already_rated_games_similarities_values})
steamdb_to_already_rated_games_similarities_df.to_csv('steamdb_to_already_rated_games_similarities.csv')

In [246]:
# find steamdb games that are in unused igdb games
more_similarities = []
for i in unique_steamdb_games['original_name'].values:
    more_similarities.append(similar(i,good_unused_igdb_games['name'].values))
more_similarities_values = np.array(more_similarities)[:,0].astype(np.float64)
more_similarities_names = np.array(more_similarities)[:,1]
more_similarities_df = pd.DataFrame({'original_name':unique_steamdb_games['original_name'].values, 'matched_name':more_similarities_names,'value':more_similarities_values})
more_similarities_df.to_csv('more_similarities.csv')

In [219]:
# adjust ratings and rating counts in combined df to use values from steamdb if game name similarity is greather than 0.95
steamdb_similarities_in_combined_df = steamdb_to_already_rated_games_similarities_df[(steamdb_to_already_rated_games_similarities_df['value'] > 0.95)]
def use_steamdb_rating(name):
    if name in (steamdb_similarities_in_combined_df['matched_name'].values):
        original_name = steamdb_similarities_in_combined_df[steamdb_similarities_in_combined_df['matched_name'] == name]['original_name'].values[0]
        return steamdb_df[steamdb_df['name'] == original_name]['rating'].values[0]
    else:
        return combined_df[combined_df['name'] == name]['rating'].values[0]
def use_steamdb_rating_counts(name):
    if name in (steamdb_similarities_in_combined_df['matched_name'].values):
        original_name = steamdb_similarities_in_combined_df[steamdb_similarities_in_combined_df['matched_name'] == name]['original_name'].values[0]
        rating_count = steamdb_df[steamdb_df['name'] == original_name]['rating_count'].values[0]
        return rating_count if rating_count > combined_df[combined_df['name'] == name]['ratings_count'].values[0] else combined_df[combined_df['name'] == name]['ratings_count'].values[0]
    else:
        return combined_df[combined_df['name'] == name]['ratings_count'].values[0]
combined_df['rating'] = combined_df['name'].apply(use_steamdb_rating)
combined_df['ratings_count'] = combined_df['name'].apply(use_steamdb_rating_counts)

In [90]:
steamdb_similarities_in_igdb = more_similarities_df[(more_similarities_df['value'] > 0.8)]

In [91]:
steamdb_games_in_igdb = steamdb_df[steamdb_df['name'].isin(steamdb_similarities_in_igdb['original_name'])]
igdb_games_in_steamdb_df = good_unused_igdb_games[good_unused_igdb_games['name'].isin(steamdb_similarities_in_igdb['matched_name'])]

In [92]:
def insert_steamdb_rating_unused(name):
    return steamdb_games_in_igdb[steamdb_games_in_igdb['name'].isin(steamdb_similarities_in_igdb[steamdb_similarities_in_igdb['matched_name'] == name]['original_name'])]['rating'].values[0]
def insert_steamdb_rating_count_unused(name):
    return steamdb_games_in_igdb[steamdb_games_in_igdb['name'].isin(steamdb_similarities_in_igdb[steamdb_similarities_in_igdb['matched_name'] == name]['original_name'])]['rating_count'].values[0]

In [ ]:
igdb_games_in_steamdb_df['total_rating'] = igdb_games_in_steamdb_df['name'].apply(insert_steamdb_rating_unused)
igdb_games_in_steamdb_df['total_rating_count'] = igdb_games_in_steamdb_df['name'].apply(insert_steamdb_rating_count_unused)
igdb_games_in_steamdb_df = igdb_games_in_steamdb_df.drop(igdb_drop_fields, axis = 1)
igdb_games_in_steamdb_df = igdb_games_in_steamdb_df.rename({'first_release_date':'release_date','id':'id_igdb','cover':'image','total_rating':'rating','total_rating_count':'ratings_count'}, axis = 1)
igdb_games_in_steamdb_df['genres'] = igdb_games_in_steamdb_df['genres'].apply(genre_id_list_to_str_list)
igdb_games_in_steamdb_df['genres'] = igdb_games_in_steamdb_df['genres'].apply(genre_change)
igdb_games_in_steamdb_df['platforms'] = igdb_games_in_steamdb_df['platforms'].apply(platform_id_to_list)
igdb_games_in_steamdb_df['platforms'] = igdb_games_in_steamdb_df['platforms'].apply(platform_change)
igdb_games_in_steamdb_df['image'] = igdb_games_in_steamdb_df['image'].apply(cover_id_to_uri)
igdb_games_in_steamdb_df['tags'] = igdb_games_in_steamdb_df['tags'].apply(tag_hash_to_tag_list)
igdb_games_in_steamdb_df['age_ratings'] = igdb_games_in_steamdb_df['age_ratings'].apply(age_rating_str_to_list)
igdb_games_in_steamdb_df['similar_games'] = igdb_games_in_steamdb_df['similar_games'].apply(similar_games_ids_to_games_list)
igdb_games_in_steamdb_df['release_date'] = pd.to_datetime(igdb_games_df_reduced['release_date']).dt.strftime('%Y-%m-%d')
combined_df = combined_df.append(igdb_games_in_steamdb_df)

In [241]:
good_unused_igdb_games = good_unused_igdb_games[(~good_unused_igdb_games['id'].isin(combined_df['id_igdb']))]
good_unused_igdb_games = good_unused_igdb_games[(~good_unused_igdb_games['platforms'].isna()) & (~good_unused_igdb_games['similar_games'].isna())]
good_unused_igdb_games = good_unused_igdb_games.drop(igdb_drop_fields, axis = 1)
good_unused_igdb_games = good_unused_igdb_games.rename({'id':'id_igdb','first_release_date':'release_date','total_rating':'rating','total_rating_count':'ratings_count'}, axis = 1)
good_unused_igdb_games['genres'] = good_unused_igdb_games['genres'].apply(genre_id_list_to_str_list)
good_unused_igdb_games['genres'] = good_unused_igdb_games['genres'].apply(genre_change)
good_unused_igdb_games['image'] = good_unused_igdb_games['image'].apply(cover_id_to_uri)
good_unused_igdb_games['tags'] = good_unused_igdb_games['tags'].apply(tag_hash_to_tag_list)
good_unused_igdb_games.columns
good_unused_igdb_games['age_ratings'] = good_unused_igdb_games['age_ratings'].apply(age_rating_str_to_list)
good_unused_igdb_games['similar_games'] = good_unused_igdb_games['similar_games'].apply(similar_games_ids_to_games_list)
good_unused_igdb_games['release_date'] = pd.to_datetime(igdb_games_df_reduced['release_date']).dt.strftime('%Y-%m-%d')
good_unused_igdb_games['platforms']
good_unused_igdb_games['platforms'] = good_unused_igdb_games['platforms'].apply(platform_id_to_list)
good_unused_igdb_games['platforms'] = good_unused_igdb_games['platforms'].apply(platform_change)

In [119]:
combined_df.to_csv('final_combined_df.csv')

In [2]:
final_df = pd.read_csv('final_combined_df.csv',index_col=[0])

C:\Users\Grayson Kornberg\AppData\Local\Temp\ipykernel_6832\2938229266.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv('final_combined_df.csv',index_col=[0])


In [8]:
final_df = final_df.reset_index().drop(['index'], axis = 1)

In [19]:
final_df = final_df.drop(['id_igdb', 'id_rawg', 'slug'], axis = 1).rename({'ratings_count':'ratingCount','summary':'description','release_date':'releaseDate','age_rating':'ageRatings','image':'imageUrl','similar_games':'similarGames'}, axis = 1)

In [21]:
final_df.to_json('data.json', orient='index')

In [22]:
final_df.to_csv('final_df_js.csv')

In [16]:
json_columns = ['index','name','description','rating','ratingCount','releaseDate','genres','platforms','tags','ageRatings','imageUrl','similarGames', 'url']